# Exploración de SET de datos

Este set de datos, voy a buscar la mejor forma de limpiar la estructura para poder dar un marco a un ETL que va a consumir la misma estructura de datos por mes y va a transformarla y generar columnas nuevas.

## set de datos

Todos los meses se realizará un excel con información de todas las facturas realizadas, tanto sean envios como retiros. En nuestro analisis le vamos a dar prioridad a los retiros.  
La limpieza se realizará en base a los retiros y el analisis en sí se va a enfocar en solo los retiros en sucursales.  
### Estructura del archivo [Columna]
Nombre -> Nombre cliente  
Documento -> Id de OARETI  
Factura -> Numero de factura  
Fecha factura -> Fecha de emisión de factura  
Pedido Magento -> Numero de pedido de E-Commers  
Tipo de producto -> Familia del producto  
Producto -> Sku o ID del producto  
Anulación -> ESTADO DEL PEDIDO   
Fecha de Movimiento -> Moviemiento de cajas SE RELACIONA DIRECTAMENTE CON FECHA DE FACTURA
Nivel 1 -> Estado despachado  
Nivel 2 -> Estado que informa sobre el estado del envío  
Conformidad -> Estado que informa sobre el estado del envío  
No Conformidad -> Estado que informa sobre el estado del envío  
Depósito -> De donde se retira la mercadería - No me brinda información de sucursales  
Hoja de Ruta -> Estado que informa sobre el estado del envío  
Fecha Entrega -> Fecha de entrega de los envios    
Nombre calle de dirección  
Altura calle de dirección  


In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
# Lectura del conjunto de datos
df = pd.read_excel(r"C:\Users\stepa\OneDrive\Escritorio\pendientesRetirosRodo\datos\datosRaw\01_enero_2025.xlsx", engine='openpyxl')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919 entries, 0 to 1918
Data columns (total 19 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Nombre                     1919 non-null   object        
 1   Documento                  1919 non-null   int64         
 2   Factura                    1919 non-null   object        
 3   Fecha factura              1919 non-null   datetime64[ns]
 4   Pedido Magento             1813 non-null   object        
 5   Tipo de producto           1919 non-null   int64         
 6   Producto                   1919 non-null   int64         
 7   Anulación                  43 non-null     object        
 8   Entrega                    1919 non-null   object        
 9   Fecha de Movimiento        1919 non-null   datetime64[ns]
 10  Nivel 1                    1919 non-null   object        
 11  Nivel 2                    947 non-null    object        
 12  Confor

## Limpieza del dataset
# Limpieza del dataset  

1- Eliminación de espacios en nombres de columnas  
2- Columnas irrelevantes  
3- División de columnas compuestas    
4- Eliminar registros de envios        
5- Normalización de formatos    
6- Registros repetidos    
7- Creacion de columnas Pre-calculadas (DIAS_PARA_RETIRO / ESTADO_RETIRO(BASADO EN UN UMBRAL TRUE O FALSE) )

Se deja calculos agregados (promedio y totales), segmentaciones dinamicas y metricas relativas a Looker 

Tengo que dividir el campo factura en FB Y FA,   
Convertir la columna PRODUCTOS a STR, para que lo cuente como Unidad.

### Eliminación de espacios en nombres de columnas

In [3]:
# Creo una copia para trabajar sobre ella, y no modificar el conjunto original
dfCopia = df.copy()

dfCopia.columns = dfCopia.columns.str.strip().str.upper().str.replace(' ', '_')
dfCopia.columns

Index(['NOMBRE', 'DOCUMENTO', 'FACTURA', 'FECHA_FACTURA', 'PEDIDO_MAGENTO',
       'TIPO_DE_PRODUCTO', 'PRODUCTO', 'ANULACIÓN', 'ENTREGA',
       'FECHA_DE_MOVIMIENTO', 'NIVEL_1', 'NIVEL_2', 'CONFORMIDAD',
       'NO_CONFORMIDAD', 'DEPÓSITO', 'HOJA_DE_RUTA', 'FECHA_ENTREGA',
       'NOMBRE_CALLE_DE_DIRECCIÓN', 'ALTURA_CALLE_DE_DIRECCIÓN'],
      dtype='object')

## Eliminar columnas que no son relevantes al analisis  

### Nuestro objetivo es únicamente analizar los pedidos que se retiran en sucursales, por ende, todas las columnas que brindan información de los envios serán eliminadas  
Podemos eliminar  
1-NIVEL_2  
2-CONFORMIDAD  
3-NO_CONFORMIDAD  
4-HOJA_DE_RUTA  
5-FECHA_ENTREGA  
6-PEDIDO MAGENTO  
7-DEPOSITO  


In [4]:
dfCopia.drop(columns=['NIVEL_2','CONFORMIDAD','NO_CONFORMIDAD','HOJA_DE_RUTA','FECHA_ENTREGA','PEDIDO_MAGENTO','DEPÓSITO'], inplace=True)
dfCopia.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1919 entries, 0 to 1918
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   NOMBRE                     1919 non-null   object        
 1   DOCUMENTO                  1919 non-null   int64         
 2   FACTURA                    1919 non-null   object        
 3   FECHA_FACTURA              1919 non-null   datetime64[ns]
 4   TIPO_DE_PRODUCTO           1919 non-null   int64         
 5   PRODUCTO                   1919 non-null   int64         
 6   ANULACIÓN                  43 non-null     object        
 7   ENTREGA                    1919 non-null   object        
 8   FECHA_DE_MOVIMIENTO        1919 non-null   datetime64[ns]
 9   NIVEL_1                    1919 non-null   object        
 10  NOMBRE_CALLE_DE_DIRECCIÓN  1919 non-null   object        
 11  ALTURA_CALLE_DE_DIRECCIÓN  447 non-null    object        
dtypes: dat

In [5]:
dfCopia.head(10)

,NOMBRE,DOCUMENTO,FACTURA,FECHA_FACTURA,TIPO_DE_PRODUCTO,PRODUCTO,ANULACIÓN,ENTREGA,FECHA_DE_MOVIMIENTO,NIVEL_1,NOMBRE_CALLE_DE_DIRECCIÓN,ALTURA_CALLE_DE_DIRECCIÓN
0,RUKY S.R.L.,30708271329,FA0501 - 2734,2025-01-02,2,340722,NaN,OARETI - 180260,2025-01-02,DESP02 - 79547,CAÑADA DE JUAN RUIZ,1120
1,CURCIO MARTIN ESTEBAN,20238389292,FA0501 - 2735,2025-01-02,2,353894,NaN,OARETI - 180438,2025-01-02,DESP14 - 22900,BELEN,545
2,MAIDANA MARCELA MERCEDES,27161374593,FA0501 - 2736,2025-01-03,24,353527,NaN,OARETI - 180503,2025-01-03,DESP02 - 79813,VILLAFAÑE BENJAMIN,2143
3,HELADERIA VIA TOSCANA S.R.L.,30714311693,FA0501 - 2737,2025-01-03,64,352786,CA0501 - 63,OARETI - 180566,2025-01-03,ZARETI - 3980,PASEO VICTORICA,470
4,E-INTRA S. R. L.,30714543381,FA0501 - 2738,2025-01-03,2,353894,NaN,OENVIO - 132056,2025-01-03,PK0002 - 98022,PABLO POGGIO,953
5,SUPERCONTROLS S A,30647361036,FA0501 - 2739,2025-01-03,43,351511,NaN,OENVIO - 132070,2025-01-03,PK0002 - 98319,Libertad,310
6,"ESTUDIO DENES, SOMODI, DENES & MAYOL",30717179184,FA0501 - 2740,2025-01-06,83,121279,NaN,OENVIO - 132554,2025-01-06,PK0002 - 98158,Diego,2000
7,DIAZ SILVIA BEATRIZ,27183995982,FA0501 - 2741,2025-01-06,43,354624,NaN,OENVIO - 132556,2025-01-06,PK0002 - 98222,MADARIAGA,1544
8,FEIJOO MARIA PILAR,27435723158,FA0501 - 2742,2025-01-06,2,347907,NaN,OENVIO - 132563,2025-01-06,PK0002 - 98446,Av. Gral. Gelly y Obes,2247
9,CRONIMETAL S.A,30601154834,FA0501 - 2743,2025-01-06,52,341565,NaN,OENVIO - 132576,2025-01-06,PK0002 - 98337,Barrio San Sebastián,área 13


## División de registros repetidos

In [6]:
#Estandarizo las columnas que tengo compuesta y limpio cualquier espacio que tengas entre valor numerico y texto
dfCopia['ENTREGA'] = (dfCopia['ENTREGA'].str.strip().str.replace(r'\s+-\s+', ' - ', regex=True))
dfCopia['FACTURA'] = (dfCopia['FACTURA'].str.strip().str.replace(r'\s+-\s+', ' - ', regex=True))



dfCopia[['NOMBRE_ENTREGA', 'ID_ENTREGA']] = (
    dfCopia['ENTREGA']
    .str.split(' - ', n=1, expand=True)  # n=1 para dividir solo en la primera ocurrencia
)

dfCopia[['TIPO_FACTURA', 'ID_FACTURA']] = (
    dfCopia['FACTURA']
    .str.split(' - ', n=1, expand=True)  # n=1 para dividir solo en la primera ocurrencia
)

dfCopia.columns




Index(['NOMBRE', 'DOCUMENTO', 'FACTURA', 'FECHA_FACTURA', 'TIPO_DE_PRODUCTO',
       'PRODUCTO', 'ANULACIÓN', 'ENTREGA', 'FECHA_DE_MOVIMIENTO', 'NIVEL_1',
       'NOMBRE_CALLE_DE_DIRECCIÓN', 'ALTURA_CALLE_DE_DIRECCIÓN',
       'NOMBRE_ENTREGA', 'ID_ENTREGA', 'TIPO_FACTURA', 'ID_FACTURA'],
      dtype='object')

In [7]:
#Elimino la columa compuesta y reodeno el dataset
dfCopia = dfCopia.drop(columns=['ENTREGA', 'FACTURA'])
dfCopia.columns


Index(['NOMBRE', 'DOCUMENTO', 'FECHA_FACTURA', 'TIPO_DE_PRODUCTO', 'PRODUCTO',
       'ANULACIÓN', 'FECHA_DE_MOVIMIENTO', 'NIVEL_1',
       'NOMBRE_CALLE_DE_DIRECCIÓN', 'ALTURA_CALLE_DE_DIRECCIÓN',
       'NOMBRE_ENTREGA', 'ID_ENTREGA', 'TIPO_FACTURA', 'ID_FACTURA'],
      dtype='object')

In [8]:
#Re ordeno las columnas
nuevo_orden = ['NOMBRE','DOCUMENTO','TIPO_FACTURA','ID_FACTURA', 'NOMBRE_ENTREGA', 'ID_ENTREGA'] + [col for col in dfCopia.columns if col not in ['NOMBRE','DOCUMENTO','TIPO_FACTURA','ID_FACTURA', 'NOMBRE_ENTREGA', 'ID_ENTREGA']]
dfCopia = dfCopia.reindex(columns=nuevo_orden)
dfCopia.columns

Index(['NOMBRE', 'DOCUMENTO', 'TIPO_FACTURA', 'ID_FACTURA', 'NOMBRE_ENTREGA',
       'ID_ENTREGA', 'FECHA_FACTURA', 'TIPO_DE_PRODUCTO', 'PRODUCTO',
       'ANULACIÓN', 'FECHA_DE_MOVIMIENTO', 'NIVEL_1',
       'NOMBRE_CALLE_DE_DIRECCIÓN', 'ALTURA_CALLE_DE_DIRECCIÓN'],
      dtype='object')

In [9]:
dfCopia.head(10)

,NOMBRE,DOCUMENTO,TIPO_FACTURA,ID_FACTURA,NOMBRE_ENTREGA,ID_ENTREGA,FECHA_FACTURA,TIPO_DE_PRODUCTO,PRODUCTO,ANULACIÓN,FECHA_DE_MOVIMIENTO,NIVEL_1,NOMBRE_CALLE_DE_DIRECCIÓN,ALTURA_CALLE_DE_DIRECCIÓN
0,RUKY S.R.L.,30708271329,FA0501,2734,OARETI,180260,2025-01-02,2,340722,NaN,2025-01-02,DESP02 - 79547,CAÑADA DE JUAN RUIZ,1120
1,CURCIO MARTIN ESTEBAN,20238389292,FA0501,2735,OARETI,180438,2025-01-02,2,353894,NaN,2025-01-02,DESP14 - 22900,BELEN,545
2,MAIDANA MARCELA MERCEDES,27161374593,FA0501,2736,OARETI,180503,2025-01-03,24,353527,NaN,2025-01-03,DESP02 - 79813,VILLAFAÑE BENJAMIN,2143
3,HELADERIA VIA TOSCANA S.R.L.,30714311693,FA0501,2737,OARETI,180566,2025-01-03,64,352786,CA0501 - 63,2025-01-03,ZARETI - 3980,PASEO VICTORICA,470
4,E-INTRA S. R. L.,30714543381,FA0501,2738,OENVIO,132056,2025-01-03,2,353894,NaN,2025-01-03,PK0002 - 98022,PABLO POGGIO,953
5,SUPERCONTROLS S A,30647361036,FA0501,2739,OENVIO,132070,2025-01-03,43,351511,NaN,2025-01-03,PK0002 - 98319,Libertad,310
6,"ESTUDIO DENES, SOMODI, DENES & MAYOL",30717179184,FA0501,2740,OENVIO,132554,2025-01-06,83,121279,NaN,2025-01-06,PK0002 - 98158,Diego,2000
7,DIAZ SILVIA BEATRIZ,27183995982,FA0501,2741,OENVIO,132556,2025-01-06,43,354624,NaN,2025-01-06,PK0002 - 98222,MADARIAGA,1544
8,FEIJOO MARIA PILAR,27435723158,FA0501,2742,OENVIO,132563,2025-01-06,2,347907,NaN,2025-01-06,PK0002 - 98446,Av. Gral. Gelly y Obes,2247
9,CRONIMETAL S.A,30601154834,FA0501,2743,OENVIO,132576,2025-01-06,52,341565,NaN,2025-01-06,PK0002 - 98337,Barrio San Sebastián,área 13


## Eliminar registros de envios

In [10]:
#Columna clave para realizar esta limpieza 'NOMBRE ENTREGA'

dfCopia = dfCopia[~dfCopia['NOMBRE_ENTREGA'].str.contains('OENVIO', case=False, na=False)]
print(f"Registros después de filtrar: {len(dfCopia)}")


Registros después de filtrar: 928


In [11]:
dfCopia['NOMBRE_ENTREGA'].value_counts()

NOMBRE_ENTREGA
OARETI    925
ODESPA      3
Name: count, dtype: int64

## Normalización de formatos de columnas

In [12]:
dfCopia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 928 entries, 0 to 1915
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   NOMBRE                     928 non-null    object        
 1   DOCUMENTO                  928 non-null    int64         
 2   TIPO_FACTURA               928 non-null    object        
 3   ID_FACTURA                 928 non-null    object        
 4   NOMBRE_ENTREGA             928 non-null    object        
 5   ID_ENTREGA                 928 non-null    object        
 6   FECHA_FACTURA              928 non-null    datetime64[ns]
 7   TIPO_DE_PRODUCTO           928 non-null    int64         
 8   PRODUCTO                   928 non-null    int64         
 9   ANULACIÓN                  23 non-null     object        
 10  FECHA_DE_MOVIMIENTO        928 non-null    datetime64[ns]
 11  NIVEL_1                    928 non-null    object        
 12  NOMBRE_CALLE

In [53]:
dfCopia['ALTURA_CALLE_DE_DIRECCIÓN'].unique()

<IntegerArray>
[1120,  545, 2143,  470, 3575, 2547, 1380, 3873, 4058, <NA>, 1399, 4926,  211,
  335,  633,   56,   71, 4449, 1185, 2245, 1375,  300,  526, 4660,  830, 1515,
 1229, 2499,  318,  138,  777, 5990,  845,  128, 4735, 1122,  395,  272,  396,
 9257,  671,   33, 5631, 2000,   51, 6900, 2049, 2447, 2683,  354, 3248, 1638,
 1150, 4265, 2311, 1529,    1, 3041, 4543,  490,  274, 1401,  367, 1011,  864,
 1762, 1917, 3346, 1669, 7304, 1360,   48, 4255, 1873, 3339, 1385, 1747, 3080,
 3385,  123, 2095,  613,  383,  113,   64,  733,  344, 3161, 4474, 2679]
Length: 90, dtype: Int64

In [13]:
dfCopia['ALTURA_CALLE_DE_DIRECCIÓN'] = (
    dfCopia['ALTURA_CALLE_DE_DIRECCIÓN']
    .astype(str)  # Convertir todo a string primero
    .str.extract(r'(\d+)', expand=False)  # Extraer solo dígitos
    .replace('nan', np.nan)  # Reemplazar strings 'nan' con NaN real
    .astype('float64')  # Convertir a float temporalmente
    .astype('Int64')  # Convertir a entero nullable (con I mayúscula)
)

In [14]:
#Se setea los datos a tipo fatetime
dfCopia['FECHA_FACTURA'] = pd.to_datetime(dfCopia['FECHA_FACTURA'], errors='coerce')
dfCopia['FECHA_DE_MOVIMIENTO'] = pd.to_datetime(dfCopia['FECHA_DE_MOVIMIENTO'], errors='coerce')


dfCopia.info()


 


<class 'pandas.core.frame.DataFrame'>
Index: 928 entries, 0 to 1915
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   NOMBRE                     928 non-null    object        
 1   DOCUMENTO                  928 non-null    int64         
 2   TIPO_FACTURA               928 non-null    object        
 3   ID_FACTURA                 928 non-null    object        
 4   NOMBRE_ENTREGA             928 non-null    object        
 5   ID_ENTREGA                 928 non-null    object        
 6   FECHA_FACTURA              928 non-null    datetime64[ns]
 7   TIPO_DE_PRODUCTO           928 non-null    int64         
 8   PRODUCTO                   928 non-null    int64         
 9   ANULACIÓN                  23 non-null     object        
 10  FECHA_DE_MOVIMIENTO        928 non-null    datetime64[ns]
 11  NIVEL_1                    928 non-null    object        
 12  NOMBRE_CALLE

In [15]:
dfCopia['ANULACIÓN'].unique()

array([nan, 'CA0501 - 63', 'CB0501 - 1869', 'CB0501 - 1875',
       'CB0501 - 1873', 'CB0501 - 1879', 'CB0501 - 1886', 'CB0501 - 1877',
       'CB0501 - 1878', 'CB0501 - 1892', 'CB0501 - 1887', 'CB0501 - 1883',
       'CB0501 - 1885', 'CB0501 - 1882', 'CB0501 - 1881', 'CB0501 - 1889',
       'CB0501 - 1894', 'CB0501 - 1888', 'CB0501 - 1891', 'CB0501 - 1902',
       'CB0501 - 1893', 'CB0501 - 1895', 'CB0501 - 1899', 'CB0501 - 1905'],
      dtype=object)

In [17]:
dfCopia['ANULACIÓN'] = dfCopia['ANULACIÓN'].fillna('Aprobada') 
dfCopia.head(10)

,NOMBRE,DOCUMENTO,TIPO_FACTURA,ID_FACTURA,NOMBRE_ENTREGA,ID_ENTREGA,FECHA_FACTURA,TIPO_DE_PRODUCTO,PRODUCTO,ANULACIÓN,FECHA_DE_MOVIMIENTO,NIVEL_1,NOMBRE_CALLE_DE_DIRECCIÓN,ALTURA_CALLE_DE_DIRECCIÓN,DIAS_RETIROS,RETIROS_TARDIOS
0,RUKY S.R.L.,30708271329,FA0501,2734,OARETI,180260,2025-01-02,2,340722,Aprobada,2025-01-02,DESP02 - 79547,CAÑADA DE JUAN RUIZ,1120,0,False
1,CURCIO MARTIN ESTEBAN,20238389292,FA0501,2735,OARETI,180438,2025-01-02,2,353894,Aprobada,2025-01-02,DESP14 - 22900,BELEN,545,0,False
2,MAIDANA MARCELA MERCEDES,27161374593,FA0501,2736,OARETI,180503,2025-01-03,24,353527,Aprobada,2025-01-03,DESP02 - 79813,VILLAFAÑE BENJAMIN,2143,0,False
3,HELADERIA VIA TOSCANA S.R.L.,30714311693,FA0501,2737,OARETI,180566,2025-01-03,64,352786,CA0501 - 63,2025-01-03,ZARETI - 3980,PASEO VICTORICA,470,0,False
10,ALBANO LUCIANO FEDERICO,20271824638,FA0501,2744,OARETI,181209,2025-01-06,2,353472,Aprobada,2025-01-06,DESP16 - 27410,COMANDANTE PIEDRA BUENA,3575,0,False
11,PABLO ESTEBAN INOUE,20224474882,FA0501,2745,OARETI,181257,2025-01-06,47,353942,Aprobada,2025-01-06,DESP09 - 135171,GRAL ALVEAR,2547,0,False
16,AGNESE ANDREA LUCILA,27173327647,FA0501,2748,OARETI,181269,2025-01-06,9,353785,Aprobada,2025-01-06,DESP15 - 45183,URIBURU,1380,0,False
18,ZALAZAR EDUARDO,20283255574,FA0501,2750,OARETI,181557,2025-01-07,47,332062,Aprobada,2025-01-07,DESP09 - 135450,AV. CORRIENTES,3873,0,False
19,ZALAZAR EDUARDO,20283255574,FA0501,2750,OARETI,181557,2025-01-07,83,325602,Aprobada,2025-01-07,DESP09 - 135450,AV. CORRIENTES,3873,0,False
20,Laura Maranesi,29365939,FB0501,72933,OARETI,180354,2024-12-20,4,346379,Aprobada,2025-01-02,DESP09 - 134322,Dr Melo,4058,13,True


## Creación de columnas Pre-calculadas

In [18]:
dfCopia['DIAS_RETIROS'] = (dfCopia['FECHA_DE_MOVIMIENTO'] - dfCopia['FECHA_FACTURA']).dt.days
dfCopia['RETIROS_TARDIOS'] = dfCopia['DIAS_RETIROS'] > 5

dfCopia = dfCopia.convert_dtypes()
dfCopia.head(10)

,NOMBRE,DOCUMENTO,TIPO_FACTURA,ID_FACTURA,NOMBRE_ENTREGA,ID_ENTREGA,FECHA_FACTURA,TIPO_DE_PRODUCTO,PRODUCTO,ANULACIÓN,FECHA_DE_MOVIMIENTO,NIVEL_1,NOMBRE_CALLE_DE_DIRECCIÓN,ALTURA_CALLE_DE_DIRECCIÓN,DIAS_RETIROS,RETIROS_TARDIOS
0,RUKY S.R.L.,30708271329,FA0501,2734,OARETI,180260,2025-01-02,2,340722,Aprobada,2025-01-02,DESP02 - 79547,CAÑADA DE JUAN RUIZ,1120,0,False
1,CURCIO MARTIN ESTEBAN,20238389292,FA0501,2735,OARETI,180438,2025-01-02,2,353894,Aprobada,2025-01-02,DESP14 - 22900,BELEN,545,0,False
2,MAIDANA MARCELA MERCEDES,27161374593,FA0501,2736,OARETI,180503,2025-01-03,24,353527,Aprobada,2025-01-03,DESP02 - 79813,VILLAFAÑE BENJAMIN,2143,0,False
3,HELADERIA VIA TOSCANA S.R.L.,30714311693,FA0501,2737,OARETI,180566,2025-01-03,64,352786,CA0501 - 63,2025-01-03,ZARETI - 3980,PASEO VICTORICA,470,0,False
10,ALBANO LUCIANO FEDERICO,20271824638,FA0501,2744,OARETI,181209,2025-01-06,2,353472,Aprobada,2025-01-06,DESP16 - 27410,COMANDANTE PIEDRA BUENA,3575,0,False
11,PABLO ESTEBAN INOUE,20224474882,FA0501,2745,OARETI,181257,2025-01-06,47,353942,Aprobada,2025-01-06,DESP09 - 135171,GRAL ALVEAR,2547,0,False
16,AGNESE ANDREA LUCILA,27173327647,FA0501,2748,OARETI,181269,2025-01-06,9,353785,Aprobada,2025-01-06,DESP15 - 45183,URIBURU,1380,0,False
18,ZALAZAR EDUARDO,20283255574,FA0501,2750,OARETI,181557,2025-01-07,47,332062,Aprobada,2025-01-07,DESP09 - 135450,AV. CORRIENTES,3873,0,False
19,ZALAZAR EDUARDO,20283255574,FA0501,2750,OARETI,181557,2025-01-07,83,325602,Aprobada,2025-01-07,DESP09 - 135450,AV. CORRIENTES,3873,0,False
20,Laura Maranesi,29365939,FB0501,72933,OARETI,180354,2024-12-20,4,346379,Aprobada,2025-01-02,DESP09 - 134322,Dr Melo,4058,13,True


In [19]:
dfCopia['DIAS_RETIROS'].value_counts()

DIAS_RETIROS
0     915
12      3
3       2
1       2
4       2
11      1
10      1
13      1
2       1
Name: count, dtype: Int64

In [22]:
# 2. Asegurar formatos adecuados para Looker Studio
dfCopia['FECHA_FACTURA'] = pd.to_datetime(dfCopia['FECHA_FACTURA']).dt.strftime('%Y-%m-%d')
dfCopia['FECHA_DE_MOVIMIENTO'] = pd.to_datetime(dfCopia['FECHA_DE_MOVIMIENTO']).dt.strftime('%Y-%m-%d')

dfCopia['PRODUCTO'] = dfCopia['PRODUCTO'].astype(str)
dfCopia['ID_FACTURA'] = dfCopia['ID_FACTURA'].astype(str) 

In [23]:
#Exporto a CSV
dfCopia.to_csv(
    r'C:\Users\stepa\OneDrive\Escritorio\pendientesRetirosRodo\datos\datosClean\lokker01-01-2025.csv',
    index=False,
    encoding='utf-8',
    date_format='%Y-%m-%d'  # Formato estándar para Looker
)